 Smart Parking System - Full End-to-End YOLOv8 Based
 Features:
 1. Automatic vehicle detection
 2. License plate recognition
 3. Dynamic slot detection (no manual slot marking)
 4. Entry/exit time logging
 5. Billing based on hourly rates


In [9]:
import cv2
import time
from ultralytics import YOLO
import easyocr
import numpy as np
from datetime import datetime, timedelta

In [10]:
# Load vehicle and license plate detection models
vehicle_model = YOLO('yolov8n.pt')  # YOLOv8n pretrained model
plate_reader = easyocr.Reader(['en'])

In [11]:
# Tracking Data
vehicles_db = {}  # vehicle_id: {plate, entry_time, exit_time, billed}
hourly_rate = 20

In [12]:
# Initialize video capture
cap = cv2.VideoCapture("video.mp4")
vehicle_id_counter = 0

In [13]:
def assign_vehicle_id(x, y):
    global vehicle_id_counter
    for vid, data in vehicles_db.items():
        px, py = data.get("last_pos", (None, None))
        if px and abs(px - x) < 50 and abs(py - y) < 50:
            return vid
    vehicle_id_counter += 1
    return f"V{vehicle_id_counter}"

In [14]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    detections = vehicle_model(frame)[0]
    for box in detections.boxes.data:
        x1, y1, x2, y2, conf, cls = map(int, box[:6])
        class_id = int(cls)
        if class_id in [2, 3, 5, 7]:  # Car, Motorcycle, Bus, Truck
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
            cx, cy = (x1+x2)//2, (y1+y2)//2
            vehicle_id = assign_vehicle_id(cx, cy)

            # Plate recognition (once per vehicle)
            vehicle_crop = frame[y1:y2, x1:x2]
            gray = cv2.cvtColor(vehicle_crop, cv2.COLOR_BGR2GRAY)
            plates = plate_reader.readtext(gray)

            plate_number = plates[0][-2] if plates else "UNKNOWN"

            if vehicle_id not in vehicles_db:
                vehicles_db[vehicle_id] = {
                    "plate": plate_number,
                    "entry_time": datetime.now(),
                    "exit_time": None,
                    "last_pos": (cx, cy),
                    "billed": False
                }
            else:
                vehicles_db[vehicle_id]["last_pos"] = (cx, cy)

            cv2.putText(frame, f"{vehicle_id} | {plate_number}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    # Display
    cv2.imshow("Smart Parking", frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 1 car, 85.1ms
Speed: 11.8ms preprocess, 85.1ms inference, 306.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 9.5ms
Speed: 2.7ms preprocess, 9.5ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 10.4ms
Speed: 3.4ms preprocess, 10.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 25.3ms
Speed: 3.9ms preprocess, 25.3ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 7.9ms
Speed: 2.2ms preprocess, 7.9ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 13.7ms
Speed: 1.9ms preprocess, 13.7ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 11.1ms
Speed: 1.9ms preprocess, 11.1ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 15.0ms
Speed: 3.2ms preprocess, 15.0ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car

Billing


In [ ]:
# Billing Summary
print("\nBilling Summary:\n")
for vid, data in vehicles_db.items():
    entry = data['entry_time']
    exit = data.get('exit_time') or datetime.now()
    duration = exit - entry
    hours = int(duration.total_seconds() // 3600) + (1 if duration.total_seconds() % 3600 > 0 else 0)
    fee = hours * hourly_rate
    print(f"Vehicle {vid} | Plate: {data['plate']} | Duration: {hours} hrs | Fee: ₹{fee}")


Billing Summary:

Vehicle V1 | Plate: Storyblocks | Duration: 1 hrs | Fee: ₹20
Vehicle V2 | Plate: UNKNOWN | Duration: 1 hrs | Fee: ₹20
Vehicle V3 | Plate: UNKNOWN | Duration: 1 hrs | Fee: ₹20
Vehicle V4 | Plate: UNKNOWN | Duration: 1 hrs | Fee: ₹20
Vehicle V5 | Plate: UNKNOWN | Duration: 1 hrs | Fee: ₹20
Vehicle V6 | Plate: UNKNOWN | Duration: 1 hrs | Fee: ₹20
Vehicle V7 | Plate: UNKNOWN | Duration: 1 hrs | Fee: ₹20
Vehicle V8 | Plate: UNKNOWN | Duration: 1 hrs | Fee: ₹20
Vehicle V9 | Plate: UNKNOWN | Duration: 1 hrs | Fee: ₹20
Vehicle V10 | Plate: UNKNOWN | Duration: 1 hrs | Fee: ₹20
Vehicle V11 | Plate: UNKNOWN | Duration: 1 hrs | Fee: ₹20
Vehicle V12 | Plate: UNKNOWN | Duration: 1 hrs | Fee: ₹20
Vehicle V13 | Plate: UNKNOWN | Duration: 1 hrs | Fee: ₹20
Vehicle V14 | Plate: UNKNOWN | Duration: 1 hrs | Fee: ₹20
Vehicle V15 | Plate: UNKNOWN | Duration: 1 hrs | Fee: ₹20
Vehicle V16 | Plate: UNKNOWN | Duration: 1 hrs | Fee: ₹20
Vehicle V17 | Plate: UNKNOWN | Duration: 1 hrs | Fee: ₹20


: 